In [71]:
# Import libraries
import os
import requests
import pandas as pd
import tensorflow_data_validation as tfdv
import tensorflow as tf
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import pprint
pp = pprint.PrettyPrinter()

## 2.1 Cargar el Dataset

In [174]:
## download the dataset
# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)

In [175]:
# Load training data into a pandas DataFrame
df = pd.read_csv('data/covertype/covertype_train.csv', index_col=False)
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


# 3. Selección de Caracteristicas

In [7]:
# Check data types
df.dtypes

Elevation                              int64
Aspect                                 int64
Slope                                  int64
Horizontal_Distance_To_Hydrology       int64
Vertical_Distance_To_Hydrology         int64
Horizontal_Distance_To_Roadways        int64
Hillshade_9am                          int64
Hillshade_Noon                         int64
Hillshade_3pm                          int64
Horizontal_Distance_To_Fire_Points     int64
Wilderness_Area                       object
Soil_Type                             object
Cover_Type                             int64
dtype: object

In [8]:
# Get only numerical columns
df_num = df.select_dtypes(include=['number'])
df_num.dtypes

Elevation                             int64
Aspect                                int64
Slope                                 int64
Horizontal_Distance_To_Hydrology      int64
Vertical_Distance_To_Hydrology        int64
Horizontal_Distance_To_Roadways       int64
Hillshade_9am                         int64
Hillshade_Noon                        int64
Hillshade_3pm                         int64
Horizontal_Distance_To_Fire_Points    int64
Cover_Type                            int64
dtype: object

In [9]:
# Split the data into features and target label
X, y = df_num.drop('Cover_Type', axis=1), df_num['Cover_Type']

# Feature selection using SelectKBest, f_classif selected as the score function since the target is categorical
X_new_features = SelectKBest(f_classif, k=8).fit(X, y)

# Create a list to indicate retained (True) or rejected (False) for each original feature
features_retained_boolean = X_new_features.get_support()

# Mapping features to their selection status
features_selection = {feature: retained for feature, retained in zip(X.columns, features_retained_boolean)}

# Check selected features are correct
X_new_features = X_new_features.get_feature_names_out(X.columns)
X_new = SelectKBest(f_classif, k=8).fit_transform(X, y)
# Display which features were retained and which were rejected
features_selection_df = pd.DataFrame(list(features_selection.items()), columns=['Columns', 'Retain'])
features_selection_df

,Columns,Retain
0,Elevation,True
1,Aspect,False
2,Slope,True
3,Horizontal_Distance_To_Hydrology,True
4,Vertical_Distance_To_Hydrology,True
5,Horizontal_Distance_To_Roadways,True
6,Hillshade_9am,True
7,Hillshade_Noon,True
8,Hillshade_3pm,False
9,Horizontal_Distance_To_Fire_Points,True


In [10]:
# Save the new dataframe to a csv file 
X = pd.DataFrame(X_new, columns=X_new_features)
df = pd.concat([X, y], axis=1)
df.to_csv('data/covertype/covertype_train.csv', index=False)

# 4. Data Pipeline

## 4.1 Configurar el contexto iterativo

In [134]:
# Enable interactive context and get examples from CSV file
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen,ExampleValidator
from tfx.v1.components import ImportSchemaGen

# Especifica la ruta al archivo de base de datos de metadatos SQLite#
metadata_db_uri = 'metadata'

context = InteractiveContext(pipeline_root = metadata_db_uri)
#context = InteractiveContext()

## 4.2 Generando Ejemplos

In [135]:
example_gen = CsvExampleGen(input_base=_data_root)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## 4.3 Estadisticas

In [136]:
# Dataset statistics
statistics_gen = StatisticsGen(
      examples=example_gen.outputs['examples']
      )
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

## 4.4 Inferir el esquema

In [137]:
# Infer schema from the statistics output
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Vertical_Distance_To_Hydrology',INT,required,,-


## 4.5 Curando el esquema

In [138]:
# Update the schema to include domain information for the categorical feature and the numerical features

from tensorflow_metadata.proto.v0 import schema_pb2

schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))
tfdv.set_domain(schema, 'Hillshade_9am',  schema_pb2.IntDomain(name='Hillshade_9am', min=0, max=255))
tfdv.set_domain(schema, 'Hillshade_Noon',  schema_pb2.IntDomain(name='Hillshade_Noon', min=0, max=255))
tfdv.set_domain(schema, 'Slope',  schema_pb2.IntDomain(name='Slope', min=0, max=90))

tfdv.display_schema(schema=schema)


,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


## 4.6 Entornos de esquema

In [139]:
# Build service data
service_df = df.sample(50)
service_df.drop('Cover_Type', axis=1, inplace=True)
service_df.to_csv('data/serving/serving_data.csv', index=False)

In [140]:
# Verify schema anomalies
serving_stats  = tfdv.generate_statistics_from_csv(data_location='./data/serving/serving_data.csv')
anomalies = tfdv.validate_statistics(statistics=serving_stats , schema=schema)
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


In [141]:
# Create schema environments

# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Specify that 'Cover_Type' feature is not in SERVING environment.
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')

serving_anomalies_with_env = tfdv.validate_statistics(
    serving_stats, schema, environment='SERVING')

tfdv.display_anomalies(serving_anomalies_with_env)

## 4.7 Genera nuevas estadísticas utilizando el esquema actualizado

In [158]:
# Dataset statistics with the updated schema
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'], schema=schema_gen.outputs['schema'])
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

In [159]:
# Save schema to file
schema_path = 'data/schema/schema.pbtxt'
tfdv.write_schema_text(schema, schema_path)

# Import the modified schema to TFX
schema_gen = ImportSchemaGen(schema_file='data/schema/schema.pbtxt')
context.run(schema_gen)
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


## 4.8 comprobar anomalias

In [160]:
# Check for anomalies with the updated schema
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)
context.show(example_validator.outputs['anomalies'])

## 4.9 Ingenieria de caracteristicas

### 4.10 Funcion de preprocesamiento

In [161]:
%%writefile _covertype_transform_module_file.py

import tensorflow as tf
import tensorflow_transform as tft

def preprocessing_fn(inputs):
    """Preprocess input DataFrame columns into transformed columns."""
    
    # No transformation for 'Cover_Type' since it's the target variable
    
    # Normalize 'Hillshade_9am' to be in [0, 1]
    hillshade_9am_scaled = tft.scale_by_min_max(inputs['Hillshade_9am'], 
                                                output_min = 0, 
                                                output_max = 1)
    
    # Normalize 'Hillshade_Noon' to be in [0, 1]
    hillshade_noon_scaled = tft.scale_by_min_max(inputs['Hillshade_Noon'], 
                                                output_min = 0, 
                                                output_max = 1)
    
    # Normalize 'Slope' to be in [0, 1]
    slope_scaled = tft.scale_by_min_max(inputs['Slope'], 
                                                output_min = 0, 
                                                output_max = 1)
    
    # The other features don't have specified min/max values for scaling.
    # z-score scaling
    elevation_z_score = tft.scale_to_z_score(inputs['Elevation'])
    distance_to_fire_points_z_score = tft.scale_to_z_score(inputs['Horizontal_Distance_To_Fire_Points'])
    distance_to_hydrology_z_score = tft.scale_to_z_score(inputs['Horizontal_Distance_To_Hydrology'])
    distance_to_roadways_z_score = tft.scale_to_z_score(inputs['Horizontal_Distance_To_Roadways'])
    vertical_distance_to_hydrology_z_score = tft.scale_to_z_score(inputs['Vertical_Distance_To_Hydrology'])
    
    # Return the transformed features
    return {
        'Cover_Type': inputs['Cover_Type'],
        'Hillshade_9am_scaled': hillshade_9am_scaled,
        'Hillshade_Noon_scaled': hillshade_noon_scaled,
        'Slope_scaled': slope_scaled,
        'Elevation_z_score': elevation_z_score,
        'Distance_To_Fire_Points_z_score': distance_to_fire_points_z_score,
        'Distance_To_Hydrology_z_score': distance_to_hydrology_z_score,
        'Distance_To_Roadways_z_score': distance_to_roadways_z_score,
        'Vertical_Distance_To_Hydrology_z_score': vertical_distance_to_hydrology_z_score
    }

Overwriting _covertype_transform_module_file.py


### 4.11 Transformar

In [162]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("_covertype_transform_module_file.py"))

context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying _covertype_transform_module_file.py -> build/lib
installing to /tmp/tmp3icjmyai
running install
running install_lib
copying build/lib/_covertype_transform_module_file.py -> /tmp/tmp3icjmyai
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp3icjmyai/tfx_user_code_Transform-0.0+21fc9e9d3fba9804eb395ecf7b11282ea0154cc4bb9ab865e3abeb63a7d59230-py3.7.egg-info
running install_scripts
creating /tmp/tmp3icjmyai/t

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


ExecutionResult(
    component_id: Transform
    execution_id: 11
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [163]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['metadata', 'transformed_metadata', 'transform_fn']

In [164]:
# Obtener el URI del artefacto de salida que representa los ejemplos transformados, que es un directorio
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Obtener la lista de archivos en este directorio (todos los archivos TFRecord comprimidos)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Crear un `TFRecordDataset` para leer estos archivos
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterar sobre el primer registro y decodificarlos.
for tfrecord in dataset.take(1):  #
    # `tfrecord` contiene un ejemplo serializado en este punto
    serialized_example = tfrecord.numpy()  # Convertir el tfrecord a un array numpy
    example = tf.train.Example()  # Crear una instancia de Example para deserializar el dato
    example.ParseFromString(serialized_example)  # Deserializar el ejemplo serializado a la estructura de datos de TensorFlow
    pp.pprint(example)  # Imprimir el ejemplo deserializado usando pretty print para una mejor legibilidad

features {
  feature {
    key: "Cover_Type"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "Distance_To_Fire_Points_z_score"
    value {
      float_list {
        value: -0.30429601669311523
      }
    }
  }
  feature {
    key: "Distance_To_Hydrology_z_score"
    value {
      float_list {
        value: -0.9478905200958252
      }
    }
  }
  feature {
    key: "Distance_To_Roadways_z_score"
    value {
      float_list {
        value: -0.8507652878761292
      }
    }
  }
  feature {
    key: "Elevation_z_score"
    value {
      float_list {
        value: 0.11900217831134796
      }
    }
  }
  feature {
    key: "Hillshade_9am_scaled"
    value {
      float_list {
        value: 0.9173228144645691
      }
    }
  }
  feature {
    key: "Hillshade_Noon_scaled"
    value {
      float_list {
        value: 0.9212598204612732
      }
    }
  }
  feature {
    key: "Slope_scaled"
    value {
      float_list {
        value: 0.106060609221

# 5 Metadatos de aprendizaje automático

## 5.1 Acceso a artefactos almacenados

In [165]:
# La URI del almacén de metadatos se puede obtener directamente de la configuración del contexto
metadata_connection_config = context.metadata_connection_config

In [166]:
from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

# Crea una instancia de MetadataStore utilizando la configuración de conexión
store = metadata_store.MetadataStore(metadata_connection_config)

In [167]:
from ml_metadata.proto import metadata_store_pb2
from ml_metadata import metadata_store
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Suponiendo que ya has inicializado el InteractiveContext como `context`
metadata_connection_config = context.metadata_connection_config

# Conectar al almacén de metadatos
store = metadata_store.MetadataStore(metadata_connection_config)

# Recuperar y mostrar todos los tipos de artefactos
artifact_types = store.get_artifact_types()
for artifact_type in artifact_types:
    print(artifact_type.name)

Examples
ExampleStatistics
Schema
ExampleAnomalies
TransformGraph
TransformCache


In [168]:
# Obtener artefactos de tipo 'Schema'
schema_artifacts = store.get_artifacts_by_type('Schema')

# Mostrar detalles de los artefactos Schema
for artifact in schema_artifacts:
    print(f"Artifact ID: {artifact.id}")
    print(f"URI: {artifact.uri}")

Artifact ID: 3
URI: metadata/SchemaGen/schema/3
Artifact ID: 5
URI: metadata/ImportSchemaGen/schema/5
Artifact ID: 10
URI: metadata/Transform/pre_transform_schema/7
Artifact ID: 12
URI: metadata/Transform/post_transform_schema/7


In [169]:
if schema_artifacts:
    first_schema_artifact = schema_artifacts[0]
    print(f"Name: {first_schema_artifact.name}")
    print(f"State: {first_schema_artifact.state}")
    print(f"Type: {first_schema_artifact.type_id}")

Name: schema:2024-03-04T00:05:39.031735
State: 2
Type: 18


## 5.2 Seguimiento de artefactos

In [170]:
def get_artifacts_details(store, type_name):
    """Obtiene detalles de artefactos por tipo y devuelve un DataFrame con su ID, Tipo y URI."""
    artifacts = store.get_artifacts_by_type(type_name)
    data = []
    
    for artifact in artifacts:
        # Añade una fila con los detalles del artefacto
        data.append({
            'Artifact ID': artifact.id,
            'Type': type_name,
            'URI': artifact.uri
        })
    
    # Convierte la lista de diccionarios a un DataFrame
    df = pd.DataFrame(data)
    return df

In [171]:
df_schema = get_artifacts_details(store, "TransformGraph")
df_schema

,Artifact ID,Type,URI
0,7,TransformGraph,metadata/Transform/transform_graph/7


## 5.3 Obtener artefactos principales

In [172]:
# Ejemplo de cómo llamar a la función para 'Examples' y mostrar el DataFrame resultante
df_examples = get_artifacts_details(store, "Examples")
df_examples

,Artifact ID,Type,URI
0,1,Examples,metadata/CsvExampleGen/examples/1
1,8,Examples,metadata/Transform/transformed_examples/7


In [173]:
# Ejemplo de cómo llamar a la función para 'Schema' y mostrar el DataFrame resultante
df_schema = get_artifacts_details(store, "Schema")
df_schema

,Artifact ID,Type,URI
0,3,Schema,metadata/SchemaGen/schema/3
1,5,Schema,metadata/ImportSchemaGen/schema/5
2,10,Schema,metadata/Transform/pre_transform_schema/7
3,12,Schema,metadata/Transform/post_transform_schema/7
